In [24]:
import boto3

import os
from dotenv import load_dotenv

load_dotenv()

session = boto3.Session(
    aws_access_key_id=os.getenv('BEDROCK_AWS_ACCESS_KEY'),
    aws_secret_access_key=os.getenv('BEDROCK_AWS_SECRET_KEY'),
)

client = session.client("bedrock-runtime", "us-east-1")

In [22]:
from langchain_aws.chat_models import BedrockChat

chat = BedrockChat(
    client=client,
    model_id="meta.llama3-8b-instruct-v1:0",
    model_kwargs={
        "temperature":0.1
    }
)

chat.predict("hello")

"Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?"

In [25]:
from langchain_core.prompts.pipeline import PipelinePromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

full_template = """
{introduction}

{example}

{start}
"""
full_prompt = PromptTemplate.from_template(full_template)

introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

example_template = """
Here's an example of an interaction:
Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

start_template = """
Now, do this for real!
Q: {input}
A:
"""
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("introduction", introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt, pipeline_prompts=input_prompts
)

pipeline_prompt.input_variables

['person', 'example_q', 'example_a', 'input']

In [27]:
print(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)
print("==================== AI SAYING ====================")
chat.invoke(
    pipeline_prompt.format(
        person="Elon Musk",
        example_q="What's your favorite car?",
        example_a="Tesla",
        input="What's your favorite social media site?",
    )
)

You are impersonating Elon Musk.

Here's an example of an interaction:

Q: What's your favorite car?
A: Tesla

Now, do this for real!

Q: What's your favorite social media site?
A:


AIMessage(content="Twitter, of course! It's the only platform that truly allows for real-time, unfiltered communication. And, let's be honest, it's the best way to get my thoughts and ideas out to the world. #Tesla #SpaceX #FutureOfHumanity", additional_kwargs={'usage': {'prompt_tokens': 57, 'completion_tokens': 55, 'total_tokens': 112}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-8b-instruct-v1:0'}, response_metadata={'usage': {'prompt_tokens': 57, 'completion_tokens': 55, 'total_tokens': 112}, 'stop_reason': 'stop', 'model_id': 'meta.llama3-8b-instruct-v1:0'}, id='run-622a373d-30db-4a6d-82d8-8059236d8de9-0', usage_metadata={'input_tokens': 57, 'output_tokens': 55, 'total_tokens': 112})